In [1]:
import os

In [2]:
%pwd

'f:\\end_to_end_flight_price_prediction\\FLIGHT_PRICE_PREDICTION\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\end_to_end_flight_price_prediction\\FLIGHT_PRICE_PREDICTION'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from Flight_Price.constants import *
from Flight_Price.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Create artifact root directory
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params["RandomForestRegressor"]
        schema = self.schema["TARGET_COLUMN"]

        # Create directories for model trainer
        create_directories([config["root_dir"]])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

        return model_evaluation_config



In [8]:
import os
import pandas as pd
from sklearn import metrics
from urllib.parse import urlparse
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def predict(self,actual,predicted):
        MAE = metrics.mean_absolute_error(actual,predicted)
        MSE = metrics.mean_squared_error(actual,predicted)
        RMSE = (np.sqrt(metrics.mean_squared_error(actual,predicted)))
        R2_Score = metrics.r2_score(actual,predicted)
        MAPE =metrics.mean_absolute_percentage_error(actual,predicted)

        return MAE,MSE,RMSE,R2_Score,MAPE
    
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (MAE,MSE, RMSE ,R2_Score,MAPE) = self.predict(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"MAE":MAE,"MSE" : MSE,"RMSE" : RMSE ,"R2_Score" : R2_Score,"MAPE" :MAPE}
        save_json(path=Path(self.config.metric_file_name), data=scores)
        
    
  

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-12-02 20:28:04,966: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-02 20:28:04,973: INFO: common: yaml file: Params.yaml loaded successfully]
[2024-12-02 20:28:04,978: INFO: common: yaml file: Schema.yaml loaded successfully]
[2024-12-02 20:28:04,981: INFO: common: created directory at: artifacts]
[2024-12-02 20:28:04,985: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-02 20:28:05,601: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
